# Common causes for migration failures 


## Hierarchyid, Geometry, Geography and sql_variant datatypes

The hierarchyid, geometry, geography and sql_variant datatypes are not currently supported.  Exclude any tables with these datatypes from the configuration migration settings blade, where you specify tables for migration.  Alternatively if it is an option, change the datatype before migration or migrate these datatypes offline. You can use this code to determine which if any of your tables have columns that have an unsupported data type.




 

In [1]:
SELECT distinct c.TABLE_NAME,c.COLUMN_NAME,c.DATA_TYPE 
from INFORMATION_SCHEMA.columns c
WHERE c.data_type in ('hierarchyid','geometry','geography', 'sql_variant' )


## Temporal Tables
Temporal tables are not supported for online migration. Exclude these tables from the configuration migration settings blade, where you specify tables for migration.  You can use this code to determine which if any of your tables are temporal:

In [4]:
select name, temporal_type, temporal_type_desc
 from sys.tables 
where temporal_type <>0


## Active Triggers
Temporal tables are not supported for online migration. Exclude these tables from the configuration migration settings blade, where you specify tables for migration.  You can use this code to determine which if any of your tables are temporal:

In [3]:
select s.name 'Schema',T.name 'Table Name',G.name 'Trigger Name'
from sys.TABLES T
join sys.triggers G
on G.parent_id = T.object_id
join sys.schemas S
on s.schema_id = t.schema_id
where is_disabled =0


## Large Object Data types
Large Object data types will require special handling. In this context, large-value data types are those that exceed the maximum row size of 8 KB. Columns larger than 32 KB may get truncated at the target. Exclude these tables from the configuration migration settings blade, where you specify tables for migration.  You can use this code to determine which if any of your columns will be affected.

In [0]:
with infoschema_CTE (Table_name, Column_name)
as (
Select TABLE_NAME, COLUMN_NAME 
from INFORMATION_SCHEMA.columns
)
select table_name, column_name,max(DATALENGTH(COLUMN_NAME)) as 'SizeInBytes'
,max(DATALENGTH(COLUMN_NAME))/1024 as 'SizeInKB'
from infoschema_CTE 
group by table_name, column_name
having (max(DATALENGTH(COLUMN_NAME))/1024) > 32
order by max(DATALENGTH(COLUMN_NAME)) desc
